In [5]:
import pandas as pd
import numpy as np
import eia
import requests
import json
import yaml
from prophet import Prophet

# Definitions
pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x)) #Limiting floats output to 3 decimal points
pd.set_option('display.max_columns', 100)    #Display upto 100 columns 
pd.set_option('display.max_rows', 100) 
%matplotlib inline

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')   

# OPTIONAL: Load the "autoreload" extension so that code can change
%load_ext autoreload

# OPTIONAL: always reload modules so that as you change code in src, it gets loaded
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
# To be able to pull modules from src folder 
import os
import sys
src_dir = os.path.join(os.getcwd(), '..', 'src')
sys.path.append(src_dir)

In [7]:
from d00_utils.const import *
from d00_utils.utils import setup_env_vars, load_yml, get_filepath

setup_env_vars()
os.environ.get("EIA_ACCESS_KEY")

2022-03-04 15:20:35,235 - Finding .env file to load entries as environment variables...
2022-03-04 15:20:35,237 - Finished loading environment variables.


'xPWAeqwYEwqmxA0hFnjsCUbcyxEG95ajpq4aX89p'

## Pulling Raw Data (JSON) and Creating Intermediate CSVs
So far, we've pulled all the raw data we're interested in and created intermediate CSVS for it - including handling of failed API requests. 

In [8]:
eia_api_ids = load_yml(EIA_API_IDS_YML_FILEPATH)
for data_type, api_ids_dict in eia_api_ids.items(): 
    print(data_type, api_ids_dict)
    print('\n')

Net_Gen_By_Fuel_MWh {'all_fuels': 'ELEC.GEN.ALL-{}-99.Q', 'coal': 'ELEC.GEN.COW-{}-99.Q', 'natural_gas': 'ELEC.GEN.NG-{}-99.Q', 'nuclear': 'ELEC.GEN.NUC-{}-99.Q', 'hydro': 'ELEC.GEN.HYC-{}-99.Q', 'other_renewables': 'ELEC.GEN.AOR-{}-99.Q', 'wind': 'ELEC.GEN.WND-{}-99.Q', 'solar_utility': 'ELEC.GEN.SUN-{}-99.Q', 'solar_all': 'ELEC.GEN.TSN-{}-99.Q', 'other': 'ELEC.GEN.OTH-{}-99.Q'}


Fuel_Consumption_BTU {'coal': 'ELEC.CONS_TOT_BTU.COW-{}-99.Q', 'natural_gas': 'ELEC.CONS_TOT_BTU.NG-{}-99.Q'}




In [9]:
# Working code so far
from d01_data.get_raw_data import load_all_data
from d02_intermediate.clean_raw_data import clean_all_data
from d03_processing.create_model_input import process_all_data

load_all_data(eia_api_ids)
clean_all_data(eia_api_ids)
process_all_data(eia_api_ids)

2022-03-04 15:20:45,595 - Invalid response for EIA Series ID: elec.gen.wnd-al-99.q with error message: invalid series_id. For key registration, documentation, and examples see https://www.eia.gov/developer/
2022-03-04 15:20:45,709 - Invalid response for EIA Series ID: elec.gen.wnd-ar-99.q with error message: invalid series_id. For key registration, documentation, and examples see https://www.eia.gov/developer/


## Model Training and Forecasting
With all the data cleaned, imputed and engineered to create the processed data which will be fed into the ML models for training, the next step is to create a framework for training and saving the time-series models for each dataset. 

In [10]:
# Sample intermediate files for testing: 
all_fuels = '../data/03_processed/Net_Gen_By_Fuel_MWh/AL/Net_Gen_By_Fuel_MWh-all_fuels.csv'
coal = '../data/03_processed/Net_Gen_By_Fuel_MWh/AL/Net_Gen_By_Fuel_MWh-coal.csv'

df_all_fuels = pd.read_csv(all_fuels)
df_coal = pd.read_csv(coal)
df_all_fuels.head()

,ds,y
0,2001-03-31,30267.126
1,2001-06-30,28933.315
2,2001-09-30,35618.737
3,2001-12-31,30525.935
4,2002-03-31,30347.665


In [11]:
df_coal.columns = ['ds', 'y']
m = Prophet()
m.fit(df_coal)

2022-03-04 15:53:36,609 - Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
2022-03-04 15:53:36,611 - Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.



Initial log joint probability = -92.1797


Iteration  1. Log joint probability =    68.6238. Improved by 160.803.
Iteration  2. Log joint probability =    127.623. Improved by 58.9992.
Iteration  3. Log joint probability =    127.625. Improved by 0.00178294.
Iteration  4. Log joint probability =    127.662. Improved by 0.0367992.
Iteration  5. Log joint probability =    127.704. Improved by 0.0421381.
Iteration  6. Log joint probability =    155.338. Improved by 27.6338.
Iteration  7. Log joint probability =    155.422. Improved by 0.0843067.
Iteration  8. Log joint probability =    155.726. Improved by 0.303883.
Iteration  9. Log joint probability =    155.848. Improved by 0.122752.
Iteration 10. Log joint probability =    155.965. Improved by 0.116853.
Iteration 11. Log joint probability =    156.014. Improved by 0.0488764.
Iteration 12. Log joint probability =     156.02. Improved by 0.00598548.
Iteration 13. Log joint probability =    156.117. Improved by 0.0971122.
Iteration 14. Log joint probability =    156.247. Improved

In [14]:
# Test function after adding to script 
from d04_modelling.create_prophet_models import train_all_models
train_all_models(eia_api_ids)

2022-03-04 15:54:39,137 - Training and Saving Prophet Models...
2022-03-04 15:54:39,138 - Training models for State: AL
2022-03-04 15:54:39,139 - Training models for Category: Net_Gen_By_Fuel_MWh
2022-03-04 15:54:39,161 - Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
2022-03-04 15:54:39,162 - Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
2022-03-04 15:54:40,771 - Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
2022-03-04 15:54:40,772 - Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.



Initial log joint probability = -91.711
Iteration  1. Log joint probability =    99.1621. Improved by 190.873.
Iteration  2. Log joint probability =    101.498. Improved by 2.33624.
Iteration  3. Log joint probability =     175.89. Improved by 74.3915.
Iteration  4. Log joint probability =    175.905. Improved by 0.0156194.
Iteration  5. Log joint probability =    175.981. Improved by 0.0753379.
Iteration  6. Log joint probability =    176.126. Improved by 0.145448.
Iteration  7. Log joint probability =    210.444. Improved by 34.3179.
Iteration  8. Log joint probability =     226.27. Improved by 15.8253.
Iteration  9. Log joint probability =    226.385. Improved by 0.114964.
Iteration 10. Log joint probability =     226.41. Improved by 0.0253536.
Iteration 11. Log joint probability =    226.939. Improved by 0.529448.
Iteration 12. Log joint probability =    227.249. Improved by 0.309778.
Iteration 13. Log joint probability =    238.459. Improved by 11.2101.
Iteration 14. Log joint pr

2022-03-04 15:54:42,087 - Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
2022-03-04 15:54:42,088 - Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.



Initial log joint probability = -92.1797
Iteration  1. Log joint probability =    68.6238. Improved by 160.803.
Iteration  2. Log joint probability =    127.623. Improved by 58.9992.
Iteration  3. Log joint probability =    127.625. Improved by 0.00178294.
Iteration  4. Log joint probability =    127.662. Improved by 0.0367992.
Iteration  5. Log joint probability =    127.704. Improved by 0.0421381.
Iteration  6. Log joint probability =    155.338. Improved by 27.6338.
Iteration  7. Log joint probability =    155.422. Improved by 0.0843067.
Iteration  8. Log joint probability =    155.726. Improved by 0.303883.
Iteration  9. Log joint probability =    155.848. Improved by 0.122752.
Iteration 10. Log joint probability =    155.965. Improved by 0.116853.
Iteration 11. Log joint probability =    156.014. Improved by 0.0488764.
Iteration 12. Log joint probability =     156.02. Improved by 0.00598548.
Iteration 13. Log joint probability =    156.117. Improved by 0.0971122.
Iteration 14. Lo

2022-03-04 15:54:43,223 - Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
2022-03-04 15:54:43,224 - Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


...


Initial log joint probability = -92.2047
Iteration  1. Log joint probability =    72.7646. Improved by 164.969.
Iteration  2. Log joint probability =    124.694. Improved by 51.929.
Iteration  3. Log joint probability =    124.703. Improved by 0.00926676.
Iteration  4. Log joint probability =    124.719. Improved by 0.0161297.
Iteration  5. Log joint probability =    124.741. Improved by 0.0222843.
Iteration  6. Log joint probability =     128.67. Improved by 3.92895.
Iteration  7. Log joint probability =    128.705. Improved by 0.0351395.
Iteration  8. Log joint probability =    128.712. Improved by 0.00706367.
Iteration  9. Log joint probability =    128.801. Improved by 0.088397.
Iteration 10. Log joint probability =    129.043. Improved by 0.242232.
Iteration 11. Log joint probability =    129.129. Improved by 0.0858131.
Iteration 12. Log joint probability =     129.25. Improved by 0.121472.
Iteration 13. Log joint probability =    129.452. Improved by 0.201255.
Iteration 14. Log 

2022-03-04 15:54:57,952 - Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
2022-03-04 15:54:57,953 - Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.



Initial log joint probability = -94.0497
Iteration  1. Log joint probability =    60.2484. Improved by 154.298.
Iteration  2. Log joint probability =    111.737. Improved by 51.4884.
Iteration  3. Log joint probability =    136.914. Improved by 25.1774.
Iteration  4. Log joint probability =    139.261. Improved by 2.34644.
Iteration  5. Log joint probability =    139.277. Improved by 0.0162344.
Iteration  6. Log joint probability =    139.297. Improved by 0.020205.
Iteration  7. Log joint probability =    139.367. Improved by 0.0695441.
Iteration  8. Log joint probability =    139.441. Improved by 0.0747506.
Iteration  9. Log joint probability =    139.515. Improved by 0.0740986.
Iteration 10. Log joint probability =    139.532. Improved by 0.0160869.
Iteration 11. Log joint probability =    139.552. Improved by 0.0206113.
Iteration 12. Log joint probability =    139.567. Improved by 0.0146565.
Iteration 13. Log joint probability =    139.674. Improved by 0.106728.
Iteration 14. Log j

2022-03-04 15:54:58,696 - Finished models for State: AR
2022-03-04 15:54:58,697 - Finished training Prophet models.



Initial log joint probability = -92.2794
Iteration  1. Log joint probability =     77.819. Improved by 170.098.
Iteration  2. Log joint probability =    87.9336. Improved by 10.1146.
Iteration  3. Log joint probability =    128.564. Improved by 40.6307.
Iteration  4. Log joint probability =    134.965. Improved by 6.40027.
Iteration  5. Log joint probability =    152.731. Improved by 17.7663.
Iteration  6. Log joint probability =    152.815. Improved by 0.0843668.
Iteration  7. Log joint probability =    152.978. Improved by 0.162601.
Iteration  8. Log joint probability =    153.049. Improved by 0.0707473.
Iteration  9. Log joint probability =    153.087. Improved by 0.0384853.
Iteration 10. Log joint probability =    153.182. Improved by 0.0953313.
Iteration 11. Log joint probability =    153.261. Improved by 0.0789262.
Iteration 12. Log joint probability =    153.644. Improved by 0.383019.
Iteration 13. Log joint probability =    153.648. Improved by 0.00389404.
Iteration 14. Log jo